# CKP8087 - Estrutura de Dados
<img  src="https://img.shields.io/badge/UFC_CKP8087-VAUX GOMES-000000?style=for-the-badge&logo=" /> <img src="https://img.shields.io/badge/Jupyter-000000?style=for-the-badge&logo=jupyter&logoColor=white" /> <img src="https://img.shields.io/badge/Python-000000?style=for-the-badge&logo=python&logoColor=white" /> <img src="https://img.shields.io/badge/PostgreSQL-000000?style=for-the-badge&logo=postgresql&logoColor=white" />

## Importando os dados

- Textos sem trava zap 
- Textos com menos de 5 palavras

In [ ]:
# Installs - Necessário porque meu Jupyter está em um Docker
!pip install pandas  
!pip install sqlalchemy
!pip install psycopg2-binary       # Drive do postgress 

# Imports
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import emoji

6227.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


6233.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


6239.52s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [84]:
# DB Config
username  = 'root'
password  = 'root'
host      = 'localhost' # NOME DO CONTAINER
port      = '5432'
database  = 'postgres' # NOME DO DATABASE

# Connection
conn = pg.connect(
    user=username,
    password=password,
    host=host,
    port=port,
    dbname=database
)

# Query
query = '''
SELECT 
    text_content_anonymous as text
FROM 
    tb_whatsapp_messages 
WHERE 
    trava_zap = false and
    array_length(regexp_split_to_array(text_content_anonymous , '\s+'), 1) < 5;
'''

# Dataframe
df = pd.read_sql(query, conn)
conn.close()

# Info
rows = df.shape[0]

# Data
print(df.shape)
df.head()

/var/folders/2_/rvh1zv7d565dmp07jlhl48340000gn/T/ipykernel_39154/1235108234.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(84008, 1)


,text
0,É sim
1,https://www.instagram.com/reel/CwV8OLXJupE/?ig...
2,https://www.instagram.com/reel/CwV8OLXJupE/?ig...
3,https://youtu.be/v7CRjTpao3g?si=OkRUBuvpFgixocH7
4,Todos Criminosos! $$$$$$$


## Árvore Digital

#### Exemplo
```py
trie = DigitalTree()
trie.add_multiple(['madri', 'barcelona', 'ma', 'manaus', 'paris', 'porto'])
trie.write_graphviz()
```

In [130]:
class DigitalTree:
    def __init__(self, path=None):
        self.root = {
          'id': 0, 'name': 'ROOT', 
          'children': {}, 'isend': False
        }
        
        self.counter = 0
        
    def __next_id(self):
      self.counter += 1
      return self.counter
    
    def add(self, word):
        pointer = self.root
        
        for l in list(word.strip().lower()):
          if l not in pointer['children']:
            pointer['children'][l] = {
              'id': self.__next_id(), 'name': l,
              'children': {}, 'isend': False
            }
            
          pointer = pointer['children'][l]
      
        pointer['isend'] = True
    
    def add_multiple(self, words):
      for word in words:
        self.add(word)
    
    def complete(self, word):
      pointer = self.root
      
      for l in list(word.strip().lower()):
        if l in pointer['children']:
          pointer = pointer['children'][l]
        else:
          break
      
      # Safety  
      if pointer == self.root:
        return []
      
      #
      stack = [(word[:-1], pointer)]
      words = []
      
      while len(stack) > 0:
        part, node = stack.pop(0)
        word_ = part + node['name']
        
        if node['isend']:
          words.append(word_)
          
        for n in node["children"].values():
          stack.append((word_, n))
          
      return words        
    
    def write_graphviz(self):
      print('digraph G {')
      print('node [shape=circle, style="rounded,filled"];')
      
      stack = [self.root]
      while len(stack) > 0:
        node = stack.pop(0)
        
        print(f'"Node {node["id"]}" [label="{node["name"]}", fillcolor="{"lightblue" if node["isend"] else "white"}"];')
        for n in node["children"].values():
          print(f'"Node {node["id"]}" -> "Node {n["id"]}";')
          stack.append(n)
          
      print('}')

#### Text treatment

In [120]:
# Remove links
def replace_links(text):
  regex = r'https?://\S+'
  return re.sub(regex, '', text)  

# Remove emojis
def replace_emojis(text):
  emoji_list = [w for w in text if w in emoji.EMOJI_DATA]

  for c in emoji_list:
    text = text.replace(c, '')
  
  return text
  
def clean_text(text):
  return re.sub(r'[^\w\s]', '', text)
  
# Aplicando funções
df['text'] = df['text'].apply(replace_links).str.strip()
df['text'] = df['text'].apply(replace_emojis).str.strip()
df['text'] = df['text'].apply(clean_text).str.strip()

df = df.replace('', np.nan)
df = df.dropna(how='all')

#### Building Tree

In [131]:
def build_tree(df):
  trie = DigitalTree()

  for t in df['text']:
    trie.add_multiple(t.split(' '))
  
  return trie

%time trie = build_tree(df[:100])

CPU times: user 3.37 ms, sys: 105 μs, total: 3.47 ms
Wall time: 3.66 ms


#### Presenting as a Graphviz Graph

In [115]:
trie.write_graphviz()

digraph G {
node [shape=circle, style="rounded,filled"];
"Node 0" [label="ROOT", fillcolor="lightblue"];
"Node 0" -> "Node 1";
"Node 0" -> "Node 2";
"Node 0" -> "Node 5";
"Node 0" -> "Node 10";
"Node 0" -> "Node 20";
"Node 0" -> "Node 23";
"Node 0" -> "Node 37";
"Node 0" -> "Node 47";
"Node 0" -> "Node 66";
"Node 0" -> "Node 69";
"Node 0" -> "Node 98";
"Node 0" -> "Node 115";
"Node 0" -> "Node 128";
"Node 0" -> "Node 143";
"Node 0" -> "Node 153";
"Node 0" -> "Node 159";
"Node 0" -> "Node 167";
"Node 0" -> "Node 181";
"Node 0" -> "Node 211";
"Node 0" -> "Node 229";
"Node 0" -> "Node 240";
"Node 0" -> "Node 275";
"Node 0" -> "Node 281";
"Node 0" -> "Node 313";
"Node 0" -> "Node 363";
"Node 0" -> "Node 514";
"Node 0" -> "Node 529";
"Node 0" -> "Node 537";
"Node 0" -> "Node 557";
"Node 1" [label="é", fillcolor="lightblue"];
"Node 2" [label="s", fillcolor="white"];
"Node 2" -> "Node 3";
"Node 2" -> "Node 224";
"Node 2" -> "Node 239";
"Node 2" -> "Node 276";
"Node 2" -> "Node 355";
"Node 5" 

![image](./files/images/graphviz.png)

#### Complete

In [132]:
text = 'type exit to end'

while text != '' and text != 'exit':
  text = input('Digite algo')
  print(text.center(len(text) + 2, ' ').center(30, '#'))
  
  for w in trie.complete(text):
    print(w)

############# b ##############
bom
boa
baba
brasil
bolsoloidi
############# j ##############
jóia
############# bo #############
bom
boa
bolsoloidi
############ exit ############
exixatamente
